In [1]:
# Notebooks
import nbimporter
import os
import sys

# Functions from src
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# Defined Functions
from utils import *

# Pandas, matplotlib, pickle, seaborn
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from statistics import mean
from collections import Counter
from imblearn.over_sampling import SMOTE, SMOTENC, ADASYN

random_state = 13 # if there is a need to fix it


In [2]:
# global variables/constants
num_trials = 30
test_size_percentage = 0.2 # for CV within train split
fixed_depth = 10



# Load Datasets

## Occutherm

In [3]:
# load TCS dataset
df_tcs = pd.read_pickle("data/occutherm/df_feature1.pkl") 
del df_tcs['Participant_No'] # original dataset contains participant number

df_tcs_train = pd.read_pickle("data/occutherm/df_feature1_train.pkl") 
df_tcs_test = pd.read_pickle("data/occutherm/df_feature1_test.pkl")

# total count for instances per class: 818
print(df_tcs_train.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1508 entries, 0 to 1507
Data columns (total 10 columns):
Temperature (Fahrenheit)       1508 non-null float64
SkinTemperature                1508 non-null float64
ClothingInsulation             1508 non-null float64
Height(cm)                     1508 non-null float64
Shoulder Circumference(cm)     1508 non-null float64
Weight(lbs)                    1508 non-null float64
Gender                         1508 non-null int64
Temperature_outside            1508 non-null float64
Humidity_outside               1508 non-null float64
Discrete Thermal Comfort_TA    1508 non-null int64
dtypes: float64(8), int64(2)
memory usage: 117.9 KB
None


## Cresh

In [4]:
df_cresh_train = pd.read_pickle("data/cresh/cresh_train.pkl")
df_cresh_test = pd.read_pickle("data/cresh/cresh_test.pkl")

# total count for instances per class: 713
print(df_cresh_train.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1159 entries, 0 to 1158
Data columns (total 11 columns):
heartRate_cozie        1159 non-null float64
humidity_sensing       1159 non-null float64
light_sensing          1159 non-null float64
noise_sensing          1159 non-null float64
temperature_sensing    1159 non-null float64
temperature_mbient     1159 non-null float64
hour_sin               1159 non-null float64
hour_cos               1159 non-null float64
day_of_week_sin        1159 non-null float64
day_of_week_cos        1159 non-null float64
thermal_cozie          1159 non-null int64
dtypes: float64(10), int64(1)
memory usage: 99.7 KB
None


## ASHRAE

In [3]:
df_ashrae_train = pd.read_pickle("data/ashrae/ashrae_train.pkl")
df_ashrae_test = pd.read_pickle("data/ashrae/ashrae_test.pkl")

print(df_ashrae_train.info())
# Number of training instances: 46477
# Number of testing instances: 19920


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46477 entries, 0 to 46476
Data columns (total 7 columns):
SET                          46477 non-null float64
Clo                          46477 non-null float64
Met                          46477 non-null float64
Air temperature (¡C)         46477 non-null float64
Relative humidity (%)        46477 non-null float64
Air velocity (m/s)           46477 non-null float64
Thermal sensation rounded    46477 non-null float64
dtypes: float64(7)
memory usage: 2.5 MB
None


# SMOTE - ADASYN

https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.SMOTE.html

In [4]:
def sample_smote_adasyn(df, dataset='occutherm', logs=True, random_state=None):
    columns = df.columns.values
    
    X = np.array(df.iloc[:, 0:df.shape[1] - 1]) # minus 1 for the comfort label
    y = np.array(df.iloc[:, -1])
    
    ########
    # SMOTE 
    ########
    if dataset == 'occutherm':
        X_res, y_res = SMOTENC(categorical_features=[6], random_state=random_state).fit_resample(X, y)
    elif dataset == 'cresh': # cresh and ashrae datasets don't have categorical features
        X_res, y_res = SMOTE(random_state=random_state).fit_resample(X, y)
    elif dataset == 'ashrae':
        X_res, y_res = SMOTE(random_state=random_state, sampling_strategy='all').fit_resample(X, y)

    # the output of .fit_resample is the real dataset with the synthetic samples
    df_x = pd.DataFrame(X_res)
    df_y = pd.DataFrame(y_res)
    df_real_synth_smote = pd.concat([df_x, df_y], axis=1)
    df_real_synth_smote.columns = columns
    df_real_synth_smote = df_real_synth_smote.sample(frac=1).reset_index(drop=True)
    # concatenate original real data to the real+synth and drop duplicates, the result will be only synth data
    df_synth_smote = pd.concat([df_real_synth_smote, df]).drop_duplicates(keep=False)   

    #########
    # ADASYN
    #########
    X_res, y_res = ADASYN(random_state=random_state).fit_resample(X, y)
    
    # the output of .fit_resample is the real dataset with the synthetic samples
    df_x = pd.DataFrame(X_res)
    df_y = pd.DataFrame(y_res)
    df_real_synth_adasyn = pd.concat([df_x, df_y], axis=1)
    df_real_synth_adasyn.columns = columns
    df_real_synth_adasyn = df_real_synth_adasyn.sample(frac=1).reset_index(drop=True) # shuffling
    # concatenate original real data to the real+synth and drop duplicates, the result will be only synth data
    df_synth_adasyn = pd.concat([df_real_synth_adasyn, df]).drop_duplicates(keep=False)
    
    if logs:
        print('Original {} train dataset shape {}'.format(dataset, Counter(y)))

        print('Resampled (SMOTE) real + synth dataset shape %s' % Counter(np.array(df_real_synth_smote.iloc[:, -1])))
        print('Resampled (SMOTE) synth dataset shape %s' % Counter(np.array(df_synth_smote.iloc[:, -1])))
        
        print('Resampled (ADASYN) train (real + synth) dataset shape %s' % Counter(np.array(df_real_synth_adasyn.iloc[:, -1])))
        print('Resampled (ADASYN) synth dataset shape %s' % Counter(np.array(df_synth_adasyn.iloc[:, -1])))
    
    return df_real_synth_smote, df_synth_smote, df_real_synth_adasyn, df_synth_adasyn


In [5]:
def evaluate_on_dataset(df_train, df_test, model='adasyn', dataset='occutherm'):
    # empty list to hold values during trials
    accgen_acc_test_list_0 = []
    accgen_acc_test_list_1 = []
    accgen_acc_test_list_2 = []
    accgen_acc_test_list_3 = []
    
    variability_list = []
    diversity_list = []

    class_acc_test_list_0 = []
    class_acc_test_list_1 = []
    class_acc_test_list_2 = []
    class_acc_test_list_3 = []
    
    class_report_rdf_list = []
    
    for i in range(0, num_trials):
        ###################################
        # Sample synthethic dataset for 'df'
        if model == 'adasyn':
            _ , _ , df_real_synth, df_synth = sample_smote_adasyn(df_train, dataset)
        elif model == 'smote':
            df_real_synth, df_synth, _ , _ = sample_smote_adasyn(df_train, dataset)
                
        ###################################
        # Variability of generated samples
        variability = evaluation_variability(df_synth)
        variability_list.append(variability)
    
        #################################################
        # Class diversity with respect to the training set
        diversity = evaluation_diversity(df_synth, df_train, baseline=False)
        diversity_list.append(diversity)

        #####################################
        # Quality on the final classification
        # use best models NB, KNN, SVM, RDF
        class_acc_test, class_acc_train, class_models, class_report_rdf = evaluation_classification(df_real_synth, df_test, rdf_depth=fixed_depth, depth_file_name='default', test_size_percentage=test_size_percentage)
        class_acc_test_list_0.append(class_acc_test[0])
        class_acc_test_list_1.append(class_acc_test[1])
        class_acc_test_list_2.append(class_acc_test[2])
        class_acc_test_list_3.append(class_acc_test[3])
        class_report_rdf_list.append(class_report_rdf)
        
        ########################
        # end of for loop trials
        print("End of {} trial".format(i + 1))
        
    # get average of trials
    accgen_acc_test = [] # no need anymore
    variability = mean(variability_list)
    diversity = mean(diversity_list)
    class_acc_test = [mean(class_acc_test_list_0), mean(class_acc_test_list_1), mean(class_acc_test_list_2), mean(class_acc_test_list_3)]
    
    #####################################
    # Saving results
    # Format is folder/<dataset_string>-<experiment_name>_<metric or model>_<test or train>_<model>.pkl
    save_pickle(accgen_acc_test, "metrics/" + dataset + "_accgen_test_" + model + "_trials.pkl")
    save_pickle(variability, "metrics/" + dataset + "_variability_" + model + "_trials.pkl")
    save_pickle(diversity, "metrics/" + dataset + "_diversity_" + model + "_trials.pkl")
    save_pickle(class_acc_test, "metrics/" + dataset + "_classification_" + model + "_trials.pkl")
    save_pickle(class_report_rdf_list, "label-metrics/" + dataset + "_class_report_" + model + "_trials.pkl")
    
    print("################################################################################")
    print("# Metrics and models for dataset {} saved!".format(dataset))
    print("################################################################################")


## Occutherm

In [8]:
evaluate_on_dataset(df_tcs_train, df_tcs_test, model='smote', dataset='occutherm')


Original occutherm train dataset shape Counter({0: 818, -1: 303, -2: 172, 1: 171, 2: 44})
Resampled (SMOTE) real + synth dataset shape Counter({2: 818, 1: 818, -1: 818, -2: 818, 0: 818})
Resampled (SMOTE) synth dataset shape Counter({2: 774, 1: 647, -2: 646, -1: 512})
Resampled (ADASYN) train (real + synth) dataset shape Counter({1: 851, -2: 823, -1: 823, 0: 818, 2: 814})
Resampled (ADASYN) synth dataset shape Counter({2: 770, 1: 680, -2: 651, -1: 518})


/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len

End of 1 trial
Original occutherm train dataset shape Counter({0: 818, -1: 303, -2: 172, 1: 171, 2: 44})
Resampled (SMOTE) real + synth dataset shape Counter({1: 818, -1: 818, 2: 818, -2: 818, 0: 818})
Resampled (SMOTE) synth dataset shape Counter({2: 774, 1: 647, -2: 646, -1: 515})
Resampled (ADASYN) train (real + synth) dataset shape Counter({1: 851, -2: 823, -1: 823, 0: 818, 2: 814})
Resampled (ADASYN) synth dataset shape Counter({2: 770, 1: 680, -2: 651, -1: 519})


/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len

End of 2 trial
Original occutherm train dataset shape Counter({0: 818, -1: 303, -2: 172, 1: 171, 2: 44})
Resampled (SMOTE) real + synth dataset shape Counter({-1: 818, -2: 818, 0: 818, 1: 818, 2: 818})
Resampled (SMOTE) synth dataset shape Counter({2: 774, 1: 647, -2: 646, -1: 512})
Resampled (ADASYN) train (real + synth) dataset shape Counter({1: 851, -1: 823, -2: 823, 0: 818, 2: 814})
Resampled (ADASYN) synth dataset shape Counter({2: 770, 1: 680, -2: 651, -1: 520})


...

/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len

End of 29 trial
Original occutherm train dataset shape Counter({0: 818, -1: 303, -2: 172, 1: 171, 2: 44})
Resampled (SMOTE) real + synth dataset shape Counter({2: 818, 0: 818, -1: 818, -2: 818, 1: 818})
Resampled (SMOTE) synth dataset shape Counter({2: 774, 1: 647, -2: 646, -1: 514})
Resampled (ADASYN) train (real + synth) dataset shape Counter({1: 851, -2: 823, -1: 823, 0: 818, 2: 814})
Resampled (ADASYN) synth dataset shape Counter({2: 770, 1: 680, -2: 651, -1: 520})


/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len

End of 30 trial
################################################################################
# Metrics and models for dataset occutherm saved!
################################################################################


/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
evaluate_on_dataset(df_tcs_train, df_tcs_test, model='adasyn', dataset='occutherm')


Original occutherm train dataset shape Counter({0: 818, -1: 303, -2: 172, 1: 171, 2: 44})
Resampled (SMOTE) real + synth dataset shape Counter({-1: 818, -2: 818, 2: 818, 0: 818, 1: 818})
Resampled (SMOTE) synth dataset shape Counter({2: 774, 1: 647, -2: 646, -1: 513})
Resampled (ADASYN) train (real + synth) dataset shape Counter({1: 851, -1: 823, -2: 823, 0: 818, 2: 814})
Resampled (ADASYN) synth dataset shape Counter({2: 770, 1: 680, -2: 651, -1: 518})


/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len

End of 1 trial
Original occutherm train dataset shape Counter({0: 818, -1: 303, -2: 172, 1: 171, 2: 44})
Resampled (SMOTE) real + synth dataset shape Counter({1: 818, -1: 818, 0: 818, -2: 818, 2: 818})
Resampled (SMOTE) synth dataset shape Counter({2: 774, 1: 647, -2: 646, -1: 515})
Resampled (ADASYN) train (real + synth) dataset shape Counter({1: 851, -1: 823, -2: 823, 0: 818, 2: 814})
Resampled (ADASYN) synth dataset shape Counter({2: 770, 1: 680, -2: 651, -1: 519})


/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len

End of 2 trial
Original occutherm train dataset shape Counter({0: 818, -1: 303, -2: 172, 1: 171, 2: 44})
Resampled (SMOTE) real + synth dataset shape Counter({-2: 818, 2: 818, 1: 818, 0: 818, -1: 818})
Resampled (SMOTE) synth dataset shape Counter({2: 774, 1: 647, -2: 646, -1: 513})
Resampled (ADASYN) train (real + synth) dataset shape Counter({1: 851, -1: 823, -2: 823, 0: 818, 2: 814})
Resampled (ADASYN) synth dataset shape Counter({2: 770, 1: 680, -2: 651, -1: 517})


...

/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len

End of 29 trial
Original occutherm train dataset shape Counter({0: 818, -1: 303, -2: 172, 1: 171, 2: 44})
Resampled (SMOTE) real + synth dataset shape Counter({-1: 818, 1: 818, 0: 818, -2: 818, 2: 818})
Resampled (SMOTE) synth dataset shape Counter({2: 774, 1: 647, -2: 646, -1: 513})
Resampled (ADASYN) train (real + synth) dataset shape Counter({1: 851, -2: 823, -1: 823, 0: 818, 2: 814})
Resampled (ADASYN) synth dataset shape Counter({2: 770, 1: 680, -2: 651, -1: 519})


/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len

End of 30 trial
################################################################################
# Metrics and models for dataset occutherm saved!
################################################################################


/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Cresh

In [10]:
evaluate_on_dataset(df_cresh_train, df_cresh_test, model='smote', dataset='cresh')


Original cresh train dataset shape Counter({10: 713, 11: 280, 9: 166})
Resampled (SMOTE) real + synth dataset shape Counter({11: 713, 9: 713, 10: 713})
Resampled (SMOTE) synth dataset shape Counter({9: 547, 11: 433})
Resampled (ADASYN) train (real + synth) dataset shape Counter({10: 713, 9: 702, 11: 685})
Resampled (ADASYN) synth dataset shape Counter({9: 536, 11: 405})


/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len

End of 1 trial
Original cresh train dataset shape Counter({10: 713, 11: 280, 9: 166})
Resampled (SMOTE) real + synth dataset shape Counter({9: 713, 10: 713, 11: 713})
Resampled (SMOTE) synth dataset shape Counter({9: 547, 11: 433})
Resampled (ADASYN) train (real + synth) dataset shape Counter({10: 713, 9: 702, 11: 685})
Resampled (ADASYN) synth dataset shape Counter({9: 536, 11: 405})


/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_st

End of 2 trial
Original cresh train dataset shape Counter({10: 713, 11: 280, 9: 166})
Resampled (SMOTE) real + synth dataset shape Counter({10: 713, 11: 713, 9: 713})
Resampled (SMOTE) synth dataset shape Counter({9: 547, 11: 433})
Resampled (ADASYN) train (real + synth) dataset shape Counter({10: 713, 9: 702, 11: 685})
Resampled (ADASYN) synth dataset shape Counter({9: 536, 11: 405})


...

/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_st

End of 28 trial
Original cresh train dataset shape Counter({10: 713, 11: 280, 9: 166})
Resampled (SMOTE) real + synth dataset shape Counter({10: 713, 9: 713, 11: 713})
Resampled (SMOTE) synth dataset shape Counter({9: 547, 11: 433})
Resampled (ADASYN) train (real + synth) dataset shape Counter({10: 713, 9: 702, 11: 685})
Resampled (ADASYN) synth dataset shape Counter({9: 536, 11: 405})


/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len

End of 29 trial
Original cresh train dataset shape Counter({10: 713, 11: 280, 9: 166})
Resampled (SMOTE) real + synth dataset shape Counter({11: 713, 10: 713, 9: 713})
Resampled (SMOTE) synth dataset shape Counter({9: 547, 11: 433})
Resampled (ADASYN) train (real + synth) dataset shape Counter({10: 713, 9: 702, 11: 685})
Resampled (ADASYN) synth dataset shape Counter({9: 536, 11: 405})
End of 30 trial
################################################################################
# Metrics and models for dataset cresh saved!
################################################################################


/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
evaluate_on_dataset(df_cresh_train, df_cresh_test, model='adasyn', dataset='cresh')


/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len

Original cresh train dataset shape Counter({10: 713, 11: 280, 9: 166})
Resampled (SMOTE) real + synth dataset shape Counter({11: 713, 10: 713, 9: 713})
Resampled (SMOTE) synth dataset shape Counter({9: 547, 11: 433})
Resampled (ADASYN) train (real + synth) dataset shape Counter({10: 713, 9: 702, 11: 685})
Resampled (ADASYN) synth dataset shape Counter({9: 536, 11: 405})


/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_st

End of 1 trial
Original cresh train dataset shape Counter({10: 713, 11: 280, 9: 166})
Resampled (SMOTE) real + synth dataset shape Counter({11: 713, 9: 713, 10: 713})
Resampled (SMOTE) synth dataset shape Counter({9: 547, 11: 433})
Resampled (ADASYN) train (real + synth) dataset shape Counter({10: 713, 9: 702, 11: 685})
Resampled (ADASYN) synth dataset shape Counter({9: 536, 11: 405})


/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_st

...

/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len

End of 29 trial
Original cresh train dataset shape Counter({10: 713, 11: 280, 9: 166})
Resampled (SMOTE) real + synth dataset shape Counter({9: 713, 10: 713, 11: 713})
Resampled (SMOTE) synth dataset shape Counter({9: 547, 11: 433})
Resampled (ADASYN) train (real + synth) dataset shape Counter({10: 713, 9: 702, 11: 685})
Resampled (ADASYN) synth dataset shape Counter({9: 536, 11: 405})


/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


End of 30 trial
################################################################################
# Metrics and models for dataset cresh saved!
################################################################################


/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matias/miniconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## ASHRAE

In [6]:
evaluate_on_dataset(df_ashrae_train, df_ashrae_test, model='smote', dataset='ashrae')


Original ashrae train dataset shape Counter({0.0: 19271, 1.0: 8872, -1.0: 7289, 2.0: 4889, -2.0: 2790, 3.0: 2161, -3.0: 1205})
Resampled (SMOTE) real + synth dataset shape Counter({-1.0: 19271, 0.0: 19271, -2.0: 19271, 3.0: 19271, 1.0: 19271, 2.0: 19271, -3.0: 19271})
Resampled (SMOTE) synth dataset shape Counter({-3.0: 17886, -2.0: 16284, 3.0: 16079, 2.0: 14085, -1.0: 11686, 1.0: 10135})
Resampled (ADASYN) train (real + synth) dataset shape Counter({2.0: 20124, 1.0: 20027, 3.0: 19847, -2.0: 19554, -1.0: 19538, -3.0: 19511, 0.0: 19271})
Resampled (ADASYN) synth dataset shape Counter({-3.0: 18166, 3.0: 16972, -2.0: 16614, 2.0: 14964, -1.0: 12052, 1.0: 10974})


/home/matias/anaconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:1449: FutureWarning: from version 0.25, pairwise_distances for metric='seuclidean' will require V to be specified if Y is passed.
  "specified if Y is passed.", FutureWarning)
/home/matias/anaconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:1449: FutureWarning: from version 0.25, pairwise_distances for metric='seuclidean' will require V to be specified if Y is passed.
  "specified if Y is passed.", FutureWarning)


End of 1 trial
Original ashrae train dataset shape Counter({0.0: 19271, 1.0: 8872, -1.0: 7289, 2.0: 4889, -2.0: 2790, 3.0: 2161, -3.0: 1205})
Resampled (SMOTE) real + synth dataset shape Counter({1.0: 19271, -2.0: 19271, -1.0: 19271, 2.0: 19271, 3.0: 19271, 0.0: 19271, -3.0: 19271})
Resampled (SMOTE) synth dataset shape Counter({-3.0: 17900, -2.0: 16292, 3.0: 16077, 2.0: 14042, -1.0: 11663, 1.0: 10125})
Resampled (ADASYN) train (real + synth) dataset shape Counter({2.0: 20124, 1.0: 20027, 3.0: 19847, -2.0: 19554, -1.0: 19538, -3.0: 19511, 0.0: 19271})
Resampled (ADASYN) synth dataset shape Counter({-3.0: 18182, 3.0: 16985, -2.0: 16617, 2.0: 14967, -1.0: 12056, 1.0: 10965})


/home/matias/anaconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:1449: FutureWarning: from version 0.25, pairwise_distances for metric='seuclidean' will require V to be specified if Y is passed.
  "specified if Y is passed.", FutureWarning)
/home/matias/anaconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:1449: FutureWarning: from version 0.25, pairwise_distances for metric='seuclidean' will require V to be specified if Y is passed.
  "specified if Y is passed.", FutureWarning)


End of 2 trial
Original ashrae train dataset shape Counter({0.0: 19271, 1.0: 8872, -1.0: 7289, 2.0: 4889, -2.0: 2790, 3.0: 2161, -3.0: 1205})
Resampled (SMOTE) real + synth dataset shape Counter({0.0: 19271, 2.0: 19271, -1.0: 19271, -2.0: 19271, -3.0: 19271, 3.0: 19271, 1.0: 19271})
Resampled (SMOTE) synth dataset shape Counter({-3.0: 17887, -2.0: 16291, 3.0: 16086, 2.0: 14056, -1.0: 11661, 1.0: 10150})
Resampled (ADASYN) train (real + synth) dataset shape Counter({2.0: 20124, 1.0: 20027, 3.0: 19847, -2.0: 19554, -1.0: 19538, -3.0: 19511, 0.0: 19271})
Resampled (ADASYN) synth dataset shape Counter({-3.0: 18152, 3.0: 16956, -2.0: 16619, 2.0: 14957, -1.0: 12026, 1.0: 10970})


...

End of 28 trial
Original ashrae train dataset shape Counter({0.0: 19271, 1.0: 8872, -1.0: 7289, 2.0: 4889, -2.0: 2790, 3.0: 2161, -3.0: 1205})
Resampled (SMOTE) real + synth dataset shape Counter({-3.0: 19271, 1.0: 19271, 3.0: 19271, -1.0: 19271, -2.0: 19271, 2.0: 19271, 0.0: 19271})
Resampled (SMOTE) synth dataset shape Counter({-3.0: 17886, -2.0: 16246, 3.0: 16089, 2.0: 14006, -1.0: 11682, 1.0: 10130})
Resampled (ADASYN) train (real + synth) dataset shape Counter({2.0: 20124, 1.0: 20027, 3.0: 19847, -2.0: 19554, -1.0: 19538, -3.0: 19511, 0.0: 19271})
Resampled (ADASYN) synth dataset shape Counter({-3.0: 18159, 3.0: 16936, -2.0: 16618, 2.0: 14985, -1.0: 12052, 1.0: 10975})


/home/matias/anaconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:1449: FutureWarning: from version 0.25, pairwise_distances for metric='seuclidean' will require V to be specified if Y is passed.
  "specified if Y is passed.", FutureWarning)
/home/matias/anaconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:1449: FutureWarning: from version 0.25, pairwise_distances for metric='seuclidean' will require V to be specified if Y is passed.
  "specified if Y is passed.", FutureWarning)


End of 29 trial
Original ashrae train dataset shape Counter({0.0: 19271, 1.0: 8872, -1.0: 7289, 2.0: 4889, -2.0: 2790, 3.0: 2161, -3.0: 1205})
Resampled (SMOTE) real + synth dataset shape Counter({3.0: 19271, 1.0: 19271, -2.0: 19271, 2.0: 19271, 0.0: 19271, -1.0: 19271, -3.0: 19271})
Resampled (SMOTE) synth dataset shape Counter({-3.0: 17907, -2.0: 16286, 3.0: 16155, 2.0: 14069, -1.0: 11685, 1.0: 10190})
Resampled (ADASYN) train (real + synth) dataset shape Counter({2.0: 20124, 1.0: 20027, 3.0: 19847, -2.0: 19554, -1.0: 19538, -3.0: 19511, 0.0: 19271})
Resampled (ADASYN) synth dataset shape Counter({-3.0: 18163, 3.0: 16963, -2.0: 16626, 2.0: 14966, -1.0: 12065, 1.0: 10966})


/home/matias/anaconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:1449: FutureWarning: from version 0.25, pairwise_distances for metric='seuclidean' will require V to be specified if Y is passed.
  "specified if Y is passed.", FutureWarning)
/home/matias/anaconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:1449: FutureWarning: from version 0.25, pairwise_distances for metric='seuclidean' will require V to be specified if Y is passed.
  "specified if Y is passed.", FutureWarning)


End of 30 trial
################################################################################
# Metrics and models for dataset ashrae saved!
################################################################################


In [7]:
evaluate_on_dataset(df_ashrae_train, df_ashrae_test, model='adasyn', dataset='ashrae')


Original ashrae train dataset shape Counter({0.0: 19271, 1.0: 8872, -1.0: 7289, 2.0: 4889, -2.0: 2790, 3.0: 2161, -3.0: 1205})
Resampled (SMOTE) real + synth dataset shape Counter({-2.0: 19271, 3.0: 19271, -1.0: 19271, 2.0: 19271, 0.0: 19271, 1.0: 19271, -3.0: 19271})
Resampled (SMOTE) synth dataset shape Counter({-3.0: 17872, -2.0: 16299, 3.0: 16102, 2.0: 14047, -1.0: 11668, 1.0: 10196})
Resampled (ADASYN) train (real + synth) dataset shape Counter({2.0: 20124, 1.0: 20027, 3.0: 19847, -2.0: 19554, -1.0: 19538, -3.0: 19511, 0.0: 19271})
Resampled (ADASYN) synth dataset shape Counter({-3.0: 18180, 3.0: 16973, -2.0: 16619, 2.0: 14961, -1.0: 12035, 1.0: 10973})


/home/matias/anaconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:1449: FutureWarning: from version 0.25, pairwise_distances for metric='seuclidean' will require V to be specified if Y is passed.
  "specified if Y is passed.", FutureWarning)
/home/matias/anaconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:1449: FutureWarning: from version 0.25, pairwise_distances for metric='seuclidean' will require V to be specified if Y is passed.
  "specified if Y is passed.", FutureWarning)


End of 1 trial
Original ashrae train dataset shape Counter({0.0: 19271, 1.0: 8872, -1.0: 7289, 2.0: 4889, -2.0: 2790, 3.0: 2161, -3.0: 1205})
Resampled (SMOTE) real + synth dataset shape Counter({0.0: 19271, 3.0: 19271, 1.0: 19271, -3.0: 19271, 2.0: 19271, -2.0: 19271, -1.0: 19271})
Resampled (SMOTE) synth dataset shape Counter({-3.0: 17875, -2.0: 16278, 3.0: 16051, 2.0: 14060, -1.0: 11696, 1.0: 10160})
Resampled (ADASYN) train (real + synth) dataset shape Counter({2.0: 20124, 1.0: 20027, 3.0: 19847, -2.0: 19554, -1.0: 19538, -3.0: 19511, 0.0: 19271})
Resampled (ADASYN) synth dataset shape Counter({-3.0: 18173, 3.0: 16984, -2.0: 16616, 2.0: 14971, -1.0: 12045, 1.0: 10969})


/home/matias/anaconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:1449: FutureWarning: from version 0.25, pairwise_distances for metric='seuclidean' will require V to be specified if Y is passed.
  "specified if Y is passed.", FutureWarning)
/home/matias/anaconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:1449: FutureWarning: from version 0.25, pairwise_distances for metric='seuclidean' will require V to be specified if Y is passed.
  "specified if Y is passed.", FutureWarning)


End of 2 trial
Original ashrae train dataset shape Counter({0.0: 19271, 1.0: 8872, -1.0: 7289, 2.0: 4889, -2.0: 2790, 3.0: 2161, -3.0: 1205})
Resampled (SMOTE) real + synth dataset shape Counter({-2.0: 19271, 1.0: 19271, -1.0: 19271, -3.0: 19271, 2.0: 19271, 3.0: 19271, 0.0: 19271})
Resampled (SMOTE) synth dataset shape Counter({-3.0: 17895, -2.0: 16292, 3.0: 16138, 2.0: 14031, -1.0: 11668, 1.0: 10156})
Resampled (ADASYN) train (real + synth) dataset shape Counter({2.0: 20124, 1.0: 20027, 3.0: 19847, -2.0: 19554, -1.0: 19538, -3.0: 19511, 0.0: 19271})
Resampled (ADASYN) synth dataset shape Counter({-3.0: 18168, 3.0: 16968, -2.0: 16624, 2.0: 14972, -1.0: 12059, 1.0: 10979})


...

End of 28 trial
Original ashrae train dataset shape Counter({0.0: 19271, 1.0: 8872, -1.0: 7289, 2.0: 4889, -2.0: 2790, 3.0: 2161, -3.0: 1205})
Resampled (SMOTE) real + synth dataset shape Counter({-2.0: 19271, -3.0: 19271, -1.0: 19271, 3.0: 19271, 2.0: 19271, 1.0: 19271, 0.0: 19271})
Resampled (SMOTE) synth dataset shape Counter({-3.0: 17897, -2.0: 16284, 3.0: 16123, 2.0: 14030, -1.0: 11636, 1.0: 10149})
Resampled (ADASYN) train (real + synth) dataset shape Counter({2.0: 20124, 1.0: 20027, 3.0: 19847, -2.0: 19554, -1.0: 19538, -3.0: 19511, 0.0: 19271})
Resampled (ADASYN) synth dataset shape Counter({-3.0: 18164, 3.0: 16961, -2.0: 16633, 2.0: 14971, -1.0: 12054, 1.0: 10957})


/home/matias/anaconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:1449: FutureWarning: from version 0.25, pairwise_distances for metric='seuclidean' will require V to be specified if Y is passed.
  "specified if Y is passed.", FutureWarning)
/home/matias/anaconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:1449: FutureWarning: from version 0.25, pairwise_distances for metric='seuclidean' will require V to be specified if Y is passed.
  "specified if Y is passed.", FutureWarning)


End of 29 trial
Original ashrae train dataset shape Counter({0.0: 19271, 1.0: 8872, -1.0: 7289, 2.0: 4889, -2.0: 2790, 3.0: 2161, -3.0: 1205})
Resampled (SMOTE) real + synth dataset shape Counter({-1.0: 19271, -3.0: 19271, -2.0: 19271, 3.0: 19271, 2.0: 19271, 0.0: 19271, 1.0: 19271})
Resampled (SMOTE) synth dataset shape Counter({-3.0: 17896, -2.0: 16282, 3.0: 16153, 2.0: 14038, -1.0: 11694, 1.0: 10146})
Resampled (ADASYN) train (real + synth) dataset shape Counter({2.0: 20124, 1.0: 20027, 3.0: 19847, -2.0: 19554, -1.0: 19538, -3.0: 19511, 0.0: 19271})
Resampled (ADASYN) synth dataset shape Counter({-3.0: 18158, 3.0: 16971, -2.0: 16623, 2.0: 14954, -1.0: 12073, 1.0: 10976})


/home/matias/anaconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:1449: FutureWarning: from version 0.25, pairwise_distances for metric='seuclidean' will require V to be specified if Y is passed.
  "specified if Y is passed.", FutureWarning)
/home/matias/anaconda3/envs/gen-methods/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:1449: FutureWarning: from version 0.25, pairwise_distances for metric='seuclidean' will require V to be specified if Y is passed.
  "specified if Y is passed.", FutureWarning)


End of 30 trial
################################################################################
# Metrics and models for dataset ashrae saved!
################################################################################
